In [1]:
!pip install faiss.cpu

  Using cached faiss_cpu-1.8.0.post1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.7 kB)
Using cached faiss_cpu-1.8.0.post1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)


In [5]:
from langchain_community.llms import Ollama
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import time

In [6]:
def execute(chain): 
    tokens = ""
    start = time.time()
    for chunk in chain.stream("Explain prompt engineering in one small paragraph"): 
        print(chunk, end='', flush=True)
        tokens += chunk
    end = time.time()
    print(f"\n\n => tokens/second : {len(tokens.split(" "))/(end-start)}")

model = Ollama(model="llama3")
execute(model)

Prompt engineering is the process of designing and refining natural language prompts to elicit specific responses or behaviors from language models, such as chatbots, virtual assistants, or other AI systems. It involves tweaking variables like wording, syntax, tone, and context to optimize the model's performance, accuracy, and understanding of the intended task or question. Effective prompt engineering can significantly improve the quality of output, reduce errors, and enhance overall conversational flow between humans and machines.

 => tokens/second : -1719926916.990373
